In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from tqdm import tqdm
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Notebook explaining the approach of how to tag NER labels to text data by mapping text files with train.csv

To see how to model this refer to my other modelling notebook: 

https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533

## Load the data

In [ ]:
train_df = pd.read_csv("/kaggle/input/feedback-prize-2021/train.csv")

In [ ]:
train_df.head()

In [ ]:
# Load the text files with corresponding ids

test_names, train_texts = [], []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/train'))[:]):
    test_names.append(f.replace('.txt', ''))
    train_texts.append(open('../input/feedback-prize-2021/train/' + f, 'r').read())
train_text_df = pd.DataFrame({'id': test_names, 'text': train_texts})
# train_texts['text'] = test_texts['text'].apply(lambda x:x.split())
train_text_df.head()

## Descripencies in train data

### as we see below length of discourse text not macthching with len of predictionstring. Further more we observe this due to 

In [ ]:
for i in train_df.iterrows():
    temp_text = i[1]['discourse_text'].split()
    temp_pred = i[1]['predictionstring'].split()
    if len(temp_text) != len(temp_pred):
        print(f"""len of discourse text : {len(temp_text)}, not macthing with len of prediction string: {len(temp_pred)}
              """)
        print(f"discourse text: {temp_text}, prediction strin: {temp_pred}")
        break

In [ ]:
### This is due to not splitting the text properly
print(train_text_df[train_text_df['id'] == "C3811E7F1750"].values[0][1].split()[302:])

In [ ]:
train_df.columns

### We will group by based on id and do some sanity checks of train.csv file with loaded text files

In [ ]:
group_df = train_df.groupby('id')['predictionstring', "discourse_text"].agg({"predictionstring":' '.join,
                                                                       "discourse_text": " ".join}).reset_index()

In [ ]:
group_df.columns

In [ ]:
# Get length of predictions string, discourse len, pred last values based on grouped data

group_df['pred_len'] = group_df['predictionstring'].apply(lambda x:len(x.split()))
group_df['discourse_len'] = group_df['discourse_text'].apply(lambda x:len(x.split()))
group_df['pred_last'] = group_df['predictionstring'].apply(lambda x: int(x.split()[-1])+1)

### Case1: There are minor cases where len of prediction labels does not match with discourse len, so we have to add extra 'O' tags at the end.

In [ ]:
group_df[group_df['pred_len'] != group_df['discourse_len']].shape[0]/train_text_df.shape[0]

In [ ]:
# group_df.sort_values('id', inplace=True)
train_text_df = train_text_df.sort_values('id').reset_index(drop=True)

In [ ]:
train_text_df.tail(10)

In [ ]:
group_df.tail(10)

In [ ]:
id_train = train_text_df['id'].values
group_df['id_train'] = id_train

In [ ]:
group_df['text_len'] = train_text_df['text'].apply(lambda x:len(x.split()))

In [ ]:
## We see text length will never match with prediction string length as some words have not annotated: compition guidelines already mentioned this

group_df[group_df['text_len'] != group_df['predictionstring']].shape[0]/train_text_df.shape[0]

### Case2: Strangely we see prediction string last index greater than text length iteself, this is due to presence of overlapping labels so we will just ignore the duplicates 


In [ ]:
group_df[group_df['text_len'] < group_df['pred_last']].shape[0]/train_text_df.shape[0]

In [ ]:
(group_df['id_train'] == group_df['id']).sum()/train_text_df.shape[0]

In [ ]:
all_entities = []
for i in tqdm(train_text_df.iterrows()):
    total = i[1]['text'].split().__len__()
#     entities = []
    entities = ["O" for i in range(total)]
    for j in train_df[train_df['id'] == i[1]['id']].iterrows():
        discourse = j[1]['discourse_type']
        list_ix = j[1]['predictionstring'].split()
        for li in list_ix[1:]:
#             print(li, entities)
            entities[int(li)] = f"I-{discourse}"
        entities[int(list_ix[0])] = f"B-{discourse}"
    all_entities.append(entities)

In [ ]:
final_df = pd.DataFrame.from_records(all_entities)

In [ ]:
final_df['id'] = train_text_df['id']

In [ ]:
train_text_df['entities'] = all_entities

In [ ]:
train_text_df.head()

In [ ]:
train_text_df.to_csv("feedback_prize_ner_tagged_data.csv", index=False)